In [1]:
from pykern import pkio
from sirepo.template.elegant import ElegantMadxConverter
from rsbeams.rsdata.SDDS import readSDDS
from rsbeams.rsstats import kinematic
from scipy import constants
import sirepo.lib
import sirepo.template
import numpy as np
import lattices

# Start
Use the MAD-X version of the IOTA lattice as processed by Sirepo

In [2]:
base_file = 'in.madx'

## MAD-X

In [3]:
model_madx = sirepo.lib.Importer('madx').parse_file(base_file)

Two changes are needed for reasonable tracking:
- Use sufficient number of kicks per element. The default of 1 will lead to incorrect results. We use 25 though the required number is probably lower.
- Increase the aperature. MAD-X uses rather small aperature by default when determining when particles are lost.

In [4]:
for cmd in model_madx.models.commands:
    if cmd._type == 'ptc_create_layout':
        cmd.nst = 25
        cmd.method = 4
    elif cmd._type == 'ptc_track':
        cmd.maxaper = "{1, 1, 1, 1, 5, 1}"

For comparison against elegant we need to remove the unsupported nllens element.
Since they are zero length we can just make them markers. The `write_files` operation will not write attributes that are not shared by the MARKER and NLLENS types so there is no need to remove them ourselves. 

In [5]:
for ele in model_madx.models.elements:
    if ele.type == 'NLLENS':
        ele.type = 'MARKER'

In [6]:
model_madx.write_files('madx/')

{}

## Convert distribution from elegant to MAD-X
Using a distribution created from elegant's matching routine in the MAD-X and impact-x simulations

In [39]:
total_energy = model_madx.models.commands[0].energy * 1e9
reference = kinematic.Converter(energy=total_energy)()


        Based on an input energy of 150000000.0 eV
        For a particle with mass: 510998.94999999995 eV/c^2
        
        velocity: 299790718.3997369 m/s
        beta: 0.9999941973181222
        gamma: 293.54267753387757
        momentum: 149999129.59771833 eV/c
        beta * gamma: 293.54097419910227
        energy: 150000000.0 eV
        kinetic energy: 149489001.05 eV
        Brho: 0.5003432394477326 T*m
        


In [45]:
elegant_distribution_file = readSDDS('bunched_beam.bunch.sdds')
elegant_distribution_file.read()
elegant_distribution = elegant_distribution_file.columns.squeeze()

In [46]:
ptc_start_str = 'ptc_start, x={x}, px={px}, y={y}, py={py}, t={t}, pt={pt};'

In [47]:
x_madx = elegant_distribution['x']
y_madx = elegant_distribution['y']

px_madx = elegant_distribution['xp'] * elegant_distribution['p'] / reference['betagamma']
py_madx = elegant_distribution['yp'] * elegant_distribution['p'] / reference['betagamma']

t_madx = elegant_distribution['t'] * -1 * constants.c
pt_madx = (kinematic.Converter(
    betagamma=elegant_distribution['p']
)(silent=True)['energy'] - total_energy) / reference['momentum']

In [48]:
with open('ptc_particles.madx', 'w') as ff:
    for x, y, px, py, t, pt in zip(x_madx, y_madx, px_madx, py_madx, t_madx, pt_madx):
        ff.write(ptc_start_str.format(x=x, y=y, px=px, py=py, t=t, pt=pt))

In [49]:
! cp ptc_particles.madx madx/

## elegant

### lattice

In [28]:
# read in newly created madx lattice that doesn't have nllens elements
path_for_elegant = 'madx/in.madx'

In [29]:
sim_type_converter = dict(
    elegant=ElegantMadxConverter,
    # opal=OpalMadxConverter,
)
sim_type_file = dict(
    elegant='elegant.ele',
    # opal='opal.in',
)

sim_type = 'elegant'
model_elegant = sirepo.lib.SimData(
    sim_type_converter[sim_type](qcall=None).from_madx(
        sirepo.lib.Importer('madx').parse_file(path_for_elegant),
    ),
    pkio.py_path(sim_type_file[sim_type]),
    sirepo.template.import_module(sim_type).LibAdapter(),
)

/home/vagrant/.pyenv/versions/py3/lib/python3.9/site-packages/sirepo/template/madx_converter.py:142:_copy_elements Unhandled element type: DIPEDGE
/home/vagrant/.pyenv/versions/py3/lib/python3.9/site-packages/sirepo/template/madx_converter.py:142:_copy_elements Unhandled element type: DIPEDGE
/home/vagrant/.pyenv/versions/py3/lib/python3.9/site-packages/sirepo/template/madx_converter.py:142:_copy_elements Unhandled element type: DIPEDGE
/home/vagrant/.pyenv/versions/py3/lib/python3.9/site-packages/sirepo/template/madx_converter.py:142:_copy_elements Unhandled element type: DIPEDGE
/home/vagrant/.pyenv/versions/py3/lib/python3.9/site-packages/sirepo/template/madx_converter.py:142:_copy_elements Unhandled element type: DIPEDGE
/home/vagrant/.pyenv/versions/py3/lib/python3.9/site-packages/sirepo/template/madx_converter.py:142:_copy_elements Unhandled element type: DIPEDGE
/home/vagrant/.pyenv/versions/py3/lib/python3.9/site-packages/sirepo/template/madx_converter.py:142:_copy_elements Unh

### Edit Lattice and Commands

For elegant it is probably better to default to converting dipoles to SBEND elements. Using CSBEND elements for a ring may be necessary but if you try to run the single-turn tracking in IOTA with default settings for CSBEND the result will be incorrect without increasing the number of kicks.

In [30]:
for ele in model_elegant.models.elements:
    if ele.type == 'CSBEND':
        ele.type = 'SBEN'

In [31]:
for cmd in model_elegant.models.commands:
    if cmd._type == 'bunched_beam':
        cmd._type = 'sdds_beam'
        cmd.input = "bunched_beam.bunch.sdds"
        cmd.center_transversely = 1
        cmd.one_random_bunch = 0
    if cmd._type == 'run_setup':
        cmd.sigma = 'run_setup.sigma.sdds'
        cmd.parameters = ''
        cmd.centroid = ''
        cmd.output = 'run_setup.output.sdds'
        cmd.p_central_mev = reference['momentum'] * 1e-6
    if cmd._type == 'twiss_output':
        cmd.filename = 'twiss_output.filename.sdds'
        cmd.matched = 1

MAD-X stores dipole edge properties in a separate element while elegant makes them properties of the dipole element

In [32]:
madx_lattice = lattices.Lattice(model_madx)
madx_iota = [ele for ele in madx_lattice.flatten_lattice('IOTA')]

In [33]:
for ele in model_elegant.models.elements:
    if ele.type == 'SBEN':
        for i, madx_ele in enumerate(madx_iota):
            if ele.name == madx_ele.name:
                assert madx_iota[i - 1].type == 'DIPEDGE'
                ele.fint = madx_iota[i - 1].fint 
                ele.hgap = madx_iota[i - 1].hgap

### Write


#### Lattice

In [34]:
model_elegant.write_files('elegant')

{'commands': local('/home/vagrant/jupyter/research/canvas/codes/iota/elegant/elegant.ele'),
 'lattice': local('/home/vagrant/jupyter/research/canvas/codes/iota/elegant/Lattice'),
 'output_files': ['run_setup.output.sdds',
  'run_setup.sigma.sdds',
  'twiss_output.filename.sdds']}

#### Distribution

In [35]:
! bunched_beam.bunch.sdds elegant/

/usr/bin/sh: line 1: bunched_beam.bunch.sdds: command not found


## ImpactX
There is no Sirepo support for ImpactX right now and ImpactX has some MAD-X lattice support so we directly use the MAD-X lattice generated above.

Some editing of the MAD-X lattice would be required to remove unsupported features and not all elements are included in the existing parser so a version of MAD-X with the Sirepo parser was used for running this simulation. See: https://github.com/cchall/impactx/tree/sirepo_madx_parser

For organizational consistency a new run directory is created with a copy of the run script (it looks like the diagnostics do not yet have an option to set output directory)

In [ ]:
# run in subdirectory 
! mkdir impactx; cp run_impactx.py impactx/run_impactx.py